In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
data = pd.read_csv('Data-Copy1.csv')
if "Unnamed: 0" in data.columns:
    data.drop(columns=["Unnamed: 0"], inplace=True)

# Define semester labels
semester_labels = [f"SEM-{i}" for i in range(1, 9)]

# Feature Engineering
data['Avg_SGPA'] = data.iloc[:, :-2].mean(axis=1)
data['SGPA_Improvement'] = data.iloc[:, 1:-2].subtract(data.iloc[:, :-3].values).mean(axis=1)
data['SGPA_Difference'] = data.iloc[:, -3] - data.iloc[:, 0]

# Split data dynamically
x_train_test = {}
y_train_test = {}
scalers = {}
mlp_models = {}

# Train models for each semester
for i in range(2, 9):
    predictors = semester_labels[: i - 1] + ["Avg_SGPA", "SGPA_Improvement", "SGPA_Difference"]
    target = f"SEM-{i}"
    
    x = data[predictors]
    y = data[target]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2)
    x_train_test[target] = (x_train, x_test)
    y_train_test[target] = (y_train, y_test)
    
    # Standardization
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    scalers[target] = scaler
    
    # Train model
    model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000, random_state=2)
    model.fit(x_train_scaled, y_train)
    mlp_models[target] = model

# Evaluation function
def evaluate_model(model, x_test_scaled, y_test, sem_label):
    y_pred = model.predict(x_test_scaled)
    print(f"Model Performance for {sem_label}:")
    print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
    print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
    print(f"R² Score: {model.score(x_test_scaled, y_test):.4f}\n")

# Get input semester
current_sem = int(input("Enter the last semester you have completed (1-7): "))

if current_sem in range(1, 8):
    # Collect SGPA for completed semesters
    print("Enter your SGPA for previous semesters one by one:")
    prev_sgpa = [float(input(f"Enter SGPA for SEM-{i}: ")) for i in range(1, current_sem + 1)]
    
    # Ask user for prediction preference
    prediction_choice = input("1:Do you want to predict only the next semester all remaining semesters? (Enter 'next' or 'all'): ").lower()
    
    # Determine which semesters to predict
    if prediction_choice == 'next':
        future_semesters = [current_sem + 1]
    else:  # Default to 'all' for any other input
        future_semesters = range(current_sem + 1, 9)
    
    # Print model evaluation for selected future semesters
    print("\nModel Evaluation:")
    for future_sem in future_semesters:
        target = f"SEM-{future_sem}"
        evaluate_model(mlp_models[target], 
                      scalers[target].transform(x_train_test[target][1]), 
                      y_train_test[target][1], 
                      target)
    
    # Predict selected future semesters
    print("\nPredictions for Future Semesters:")
    
    # Make a copy of the previous SGPAs to update as we make predictions
    all_sgpa = prev_sgpa.copy()
    
    for future_sem in future_semesters:
        target = f"SEM-{future_sem}"
        
        # Get only the required previous semesters for this prediction
        required_prev_sgpa = all_sgpa[:future_sem-1]
        
        # Compute derived features
        avg_sgpa = np.mean(required_prev_sgpa)
        sgpa_improvement = np.mean(np.diff(required_prev_sgpa)) if len(required_prev_sgpa) > 1 else 0
        sgpa_difference = required_prev_sgpa[-1] - required_prev_sgpa[0] if len(required_prev_sgpa) > 1 else 0
        
        # Create input features array
        new_input_extended = np.array(required_prev_sgpa + [avg_sgpa, sgpa_improvement, sgpa_difference]).reshape(1, -1)
        
        # Standardize and Predict
        new_input_scaled = scalers[target].transform(new_input_extended)
        prediction = mlp_models[target].predict(new_input_scaled)
        predicted_sgpa = round(prediction[0], 2)
        
        print(f"Predicted SGPA for {target}: {predicted_sgpa}")
        
        # Add the prediction to all_sgpa for use in subsequent predictions
        all_sgpa.append(predicted_sgpa)
else:
    print("Invalid choice. Please enter a number between 1 and 7.")

Enter the last semester you have completed (1-7):  4


Enter your SGPA for previous semesters one by one:


Enter SGPA for SEM-1:  8.06
Enter SGPA for SEM-2:  7.79
Enter SGPA for SEM-3:  8.32
Enter SGPA for SEM-4:  7.89
1:Do you want to predict only the next semester all remaining semesters? (Enter 'next' or 'all'):  next



Model Evaluation:
Model Performance for SEM-5:
MAE: 0.2309
MSE: 0.0919
R² Score: 0.7694


Predictions for Future Semesters:
Predicted SGPA for SEM-5: 8.13


C:\Users\Noyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
